![image](how.png)

**"Should I Stay or Should i Go (from the Hospital)"**

In [71]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from sklearn import linear_model
# la otra forma, pero lo mismo: from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

In [72]:
df = pd.read_csv("train_set.csv")

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 18 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       100000 non-null  int64  
 1   1       100000 non-null  int64  
 2   2       100000 non-null  object 
 3   3       100000 non-null  int64  
 4   4       100000 non-null  object 
 5   5       100000 non-null  int64  
 6   6       100000 non-null  object 
 7   7       100000 non-null  object 
 8   8       100000 non-null  object 
 9   9       99967 non-null   float64
 10  10      100000 non-null  int64  
 11  11      98517 non-null   float64
 12  12      100000 non-null  object 
 13  13      100000 non-null  object 
 14  14      100000 non-null  int64  
 15  15      100000 non-null  object 
 16  16      100000 non-null  float64
 17  17      100000 non-null  object 
dtypes: float64(3), int64(6), object(9)
memory usage: 13.7+ MB


In [74]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,161528,6,a,6,X,2,gynecology,R,F,4.0,45810,2.0,Urgent,Moderate,2,21-30,2817.0,0-10
1,159472,23,a,6,X,4,gynecology,Q,F,2.0,128565,15.0,Trauma,Moderate,4,51-60,4498.0,21-30
2,309765,2,c,5,Z,2,anesthesia,S,F,3.0,46565,5.0,Urgent,Moderate,2,71-80,4573.0,11-20
3,279614,32,f,9,Y,3,gynecology,S,B,4.0,124546,6.0,Emergency,Moderate,4,11-20,7202.0,51-60
4,147791,14,a,1,X,3,gynecology,S,E,2.0,22729,8.0,Urgent,Moderate,2,51-60,3398.0,51-60


In [75]:
# Obtengo un daframe que contiene solamente la columna 0 que luego necesitare que el modelo aprenda
df_0 = df['0']

In [76]:
df_p = df.copy()
df_p['9'] = df_p['9'].interpolate(method='nearest')
df_p['9'].value_counts()

2.0    38947
3.0    34560
4.0    18100
1.0     8393
Name: 9, dtype: int64

In [77]:
df_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 18 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       100000 non-null  int64  
 1   1       100000 non-null  int64  
 2   2       100000 non-null  object 
 3   3       100000 non-null  int64  
 4   4       100000 non-null  object 
 5   5       100000 non-null  int64  
 6   6       100000 non-null  object 
 7   7       100000 non-null  object 
 8   8       100000 non-null  object 
 9   9       100000 non-null  float64
 10  10      100000 non-null  int64  
 11  11      98517 non-null   float64
 12  12      100000 non-null  object 
 13  13      100000 non-null  object 
 14  14      100000 non-null  int64  
 15  15      100000 non-null  object 
 16  16      100000 non-null  float64
 17  17      100000 non-null  object 
dtypes: float64(3), int64(6), object(9)
memory usage: 13.7+ MB


In [8]:
# Los Nan en la columna 11 son casi 1500 y como la columna es el código de la ciudad de origen del paciente, decido no usarla al igual que la 4, 10, 12, 16


In [36]:
# Convierto todos los datos que son float a integer en columnas que me interesan
#df_p[['9', '16']] = df_p[['9','16']].astype(int)

In [78]:
df_p.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,161528,6,a,6,X,2,gynecology,R,F,4.0,45810,2.0,Urgent,Moderate,2,21-30,2817.0,0-10
1,159472,23,a,6,X,4,gynecology,Q,F,2.0,128565,15.0,Trauma,Moderate,4,51-60,4498.0,21-30
2,309765,2,c,5,Z,2,anesthesia,S,F,3.0,46565,5.0,Urgent,Moderate,2,71-80,4573.0,11-20
3,279614,32,f,9,Y,3,gynecology,S,B,4.0,124546,6.0,Emergency,Moderate,4,11-20,7202.0,51-60
4,147791,14,a,1,X,3,gynecology,S,E,2.0,22729,8.0,Urgent,Moderate,2,51-60,3398.0,51-60


In [79]:
#Creo un nuevo df que contendrá unicamente las columnas que me son utilies para el estudio.
df_util = df_p[['1','2', '3','5', '6', '7','8','9','13','14','15','17']]

In [80]:
df_util.head()

,1,2,3,5,6,7,8,9,13,14,15,17
0,6,a,6,2,gynecology,R,F,4.0,Moderate,2,21-30,0-10
1,23,a,6,4,gynecology,Q,F,2.0,Moderate,4,51-60,21-30
2,2,c,5,2,anesthesia,S,F,3.0,Moderate,2,71-80,11-20
3,32,f,9,3,gynecology,S,B,4.0,Moderate,4,11-20,51-60
4,14,a,1,3,gynecology,S,E,2.0,Moderate,2,51-60,51-60


In [84]:
# Selecciono las columnas con valores categóricos.
df_categorical_no_numbers = df_util[df_util.select_dtypes('object').columns]
df_categorical_no_numbers.head()

,2,6,7,8,13,15,17
0,a,gynecology,R,F,Moderate,21-30,0-10
1,a,gynecology,Q,F,Moderate,51-60,21-30
2,c,anesthesia,S,F,Moderate,71-80,11-20
3,f,gynecology,S,B,Moderate,11-20,51-60
4,a,gynecology,S,E,Moderate,51-60,51-60


In [81]:
df_others = df_util.select_dtypes(exclude=['object'])
df_others.head()

,1,3,5,9,14
0,6,6,2,4.0,2
1,23,6,4,2.0,4
2,2,5,2,3.0,2
3,32,9,3,4.0,4
4,14,1,3,2.0,2


In [85]:
# Aplico normalización a las columnas numericas que están en el df_others
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_others_normalized = pd.DataFrame(
    data=scaler.fit_transform(df_others.values), 
    columns=df_others.columns, 
    index=df_others.index)

In [86]:
df_others_normalized.head() 

,1,3,5,9,14
0,0.161290,0.416667,0.083333,1.000000,0.0625
1,0.709677,0.416667,0.166667,0.333333,0.1250
2,0.032258,0.333333,0.083333,0.666667,0.0625
3,1.000000,0.666667,0.125000,1.000000,0.1250
4,0.419355,0.000000,0.125000,0.333333,0.0625


In [87]:
# Dataframe con valores categoricos encode, valores numéricos normalizados y columnas ordenadas.
df_estudio = pd.concat([df_categorical_no_numbers,df_others_normalized], axis=1)
df_estudio = df_estudio[['1','2', '3','5', '6', '7','8','9','13','14','15','17']]
df_estudio.head()

,1,2,3,5,6,7,8,9,13,14,15,17
0,0.161290,a,0.416667,0.083333,gynecology,R,F,1.000000,Moderate,0.0625,21-30,0-10
1,0.709677,a,0.416667,0.166667,gynecology,Q,F,0.333333,Moderate,0.1250,51-60,21-30
2,0.032258,c,0.333333,0.083333,anesthesia,S,F,0.666667,Moderate,0.0625,71-80,11-20
3,1.000000,f,0.666667,0.125000,gynecology,S,B,1.000000,Moderate,0.1250,11-20,51-60
4,0.419355,a,0.000000,0.125000,gynecology,S,E,0.333333,Moderate,0.0625,51-60,51-60


In [100]:
# esto es necesario para evitar futuros errores pero si quitaráis las columnas con Nan no haría falta
# Nan no vale pero 0 sí
df_estudio = df_estudio.dropna()
# esto quita filas y por tanto no se lo podremos aplicar al dataset de predicción

#### 1. Definir X e y

In [101]:
X = df_estudio[['1','2', '3','5', '6', '7','8','9','13','14','15']].values
X

array([[0.16129032258064516, 'a', 0.4166666666666667, ..., 'Moderate',
        0.0625, '21-30'],
       [0.7096774193548387, 'a', 0.4166666666666667, ..., 'Moderate',
        0.125, '51-60'],
       [0.03225806451612903, 'c', 0.3333333333333333, ..., 'Moderate',
        0.0625, '71-80'],
       ...,
       [0.16129032258064516, 'a', 0.4166666666666667, ..., 'Minor',
        0.0625, '31-40'],
       [1.0, 'f', 0.6666666666666666, ..., 'Moderate', 0.09375, '41-50'],
       [0.7741935483870968, 'e', 0.0, ..., 'Moderate', 0.0625, '41-50']],
      dtype=object)

In [102]:
X.shape

(100000, 11)

In [109]:
y = np.array(df_estudio['17'])
y

array(['0-10', '21-30', '11-20', ..., '31-40', '11-20', '41-50'],
      dtype=object)

#### 2. Dividir X_train, X_test, y_train, y_test

In [110]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [111]:
X_train

array([[0.16129032258064516, 'a', 0.4166666666666667, ..., 'Extreme',
        0.125, '31-40'],
       [0.8709677419354839, 'b', 0.8333333333333333, ..., 'Extreme',
        0.0625, '71-80'],
       [0.32258064516129037, 'b', 0.08333333333333333, ..., 'Moderate',
        0.125, '51-60'],
       ...,
       [0.25806451612903225, 'd', 0.3333333333333333, ..., 'Moderate',
        0.125, '41-50'],
       [0.3548387096774194, 'a', 0.6666666666666666, ..., 'Moderate',
        0.0625, '21-30'],
       [0.22580645161290322, 'c', 0.16666666666666669, ..., 'Moderate',
        0.0625, '71-80']], dtype=object)

#### 3. Asignar el modelo (vacío) a una variable
Aquí meteríais los parámetros

In [112]:
model= linear_model.LogisticRegression

#### 4. Entrenar el modelo con X_train, y_train

In [113]:
model.fit(X_train, y_train)

TypeError: fit() missing 1 required positional argument: 'y'

#### 5. Predecir con el modelo ya entrenado con X_test/to_pred

In [33]:
predictions = model.predict(X_test)
print(predictions)

[2 5 1 ... 2 2 1]


#### 6. Sacar métricas, valorar el modelo 

In [34]:
model.score(X_test,y_test)

0.3417

## Una vez listo el modelo, toca predecir con el dataset de predicción 

In [35]:
to_pred = pd.read_csv("test_set.csv")

In [53]:
# la primera columna es id_case, deberéis quitarla para hacer la predicción si la habéis quitado para entrenar
# pero guardarla para incluirla en la submission

# IMPORTANTE: APLICAR LO MISMO A ESTOS DATOS QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO
# SI EL ARRAY CON EL QUE HICISTEIS .fit() ERA DE 4 COLUMNAS, PARA .predict() DEBEN SER LAS MISMAS
# SI AL ARRAY CON EL QUE HICISTEIS .fit() LO NORMALIZASTEIS, PARA .predict() DEBÉIS NORMALIZARLO
# TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 133744 FILAS
# SI O SI

to_pred.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,314114,19,a,7,Y,4,gynecology,S,C,2.0,59278,8.0,Emergency,Minor,2,41-50,4778.0
1,208989,15,c,5,Z,3,gynecology,S,F,2.0,102253,15.0,Trauma,Moderate,3,31-40,5734.0
2,305872,17,e,1,X,4,gynecology,R,E,4.0,5828,4.0,Emergency,Minor,3,71-80,5064.0
3,266099,3,c,3,Z,4,TB & Chest disease,R,A,2.0,56642,9.0,Urgent,Extreme,4,31-40,3254.0
4,13228,6,a,6,X,4,gynecology,R,F,1.0,116266,8.0,Emergency,Minor,3,21-30,4639.0


In [54]:
# Reservo columna cero de to_pred
to_pred_0 = to_pred['0']

In [55]:
# Relleno Nan columna 9
to_pred['9'] = to_pred['9'].interpolate(method='nearest')

In [56]:
# para evitar este error ValueError: Input contains NaN, infinity or a value too large for dtype('float64').
to_pred = to_pred.fillna(0)

In [57]:
to_pred_estudio = to_pred[['1','2', '3','5', '6', '7','8','9','13','14','15']]

In [58]:
# Encode all columns, pero solo lo aplico a las categóricas.
to_pred_estudio_categorical_no_numbers = to_pred_estudio[to_pred_estudio.select_dtypes('object').columns].apply(LabelEncoder().fit_transform)
to_pred_estudio_categorical_no_numbers.head()

,2,6,7,8,13,15
0,0,2,3,2,1,4
1,2,2,3,5,2,3
2,4,2,2,4,1,7
3,2,0,2,0,0,3
4,0,2,2,5,1,2


In [ ]:
# Aplico normalización solo a las columnas con valores numérios

In [59]:
#Creo df con las columnas sobre la que normalizare
to_pred_estudio_others = to_pred_estudio.select_dtypes(exclude=['object'])
# Normalizo
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
to_pred_estudio_others_normalized = pd.DataFrame(
    data=scaler.fit_transform(to_pred_estudio_others.values), 
    columns=to_pred_estudio_others.columns, 
    index=to_pred_estudio_others.index)


In [60]:
# Dataframe con valores categoricos encode, valores numéricos normalizados y columnas ordenadas.
to_pred_final = pd.concat([to_pred_estudio_categorical_no_numbers, to_pred_estudio_others], axis=1)
to_pred_final = to_pred_final[['1','2', '3','5', '6', '7','8','9','13','14','15']]
to_pred_final.head()

,1,2,3,5,6,7,8,9,13,14,15
0,19,0,7,4,2,3,2,2.0,1,2,4
1,15,2,5,3,2,3,5,2.0,2,3,3
2,17,4,1,4,2,2,4,4.0,1,3,7
3,3,2,3,4,0,2,0,2.0,0,4,3
4,6,0,6,4,2,2,5,1.0,1,3,2


In [61]:
predictions_submit = model. predict(to_pred_final[['1','2', '3','5', '6', '7','8','9','13','14','15']])
predictions_submit

array([4, 5, 4, ..., 5, 4, 5], dtype=int64)

In [65]:
from sklearn.preprocessing import LabelEncoder

In [66]:
pred_submit_code= le_17.inverse_transform(predictions_submit)

In [67]:
pred_submit_code

array([4, 5, 4, ..., 5, 4, 5], dtype=int64)

In [36]:
submission_RL = pd.DataFrame({'id': to_pred['0'], 'days': predictions_submit})
submission_RL

,id,days
0,314114,4
1,208989,5
2,305872,4
3,266099,5
4,13228,5
...,...,...
133739,318155,5
133740,144850,4
133741,180676,5
133742,39933,4


In [37]:
sample = pd.read_csv("sample_submission.csv")

In [37]:
sample.shape

(133744, 2)

In [38]:
if submission_RL.shape == sample.shape:
    if submission_RL.columns.all() == sample.columns.all():
        if submission_RL.id.all() == sample.id.all():
            print("you're ready to submit!")
            submission_RL.to_csv("to_submit_RL.csv", index = False)
            # ¡¡¡¡¡¡¡ADD INDEX = FALSE!!!!!!!!!

you're ready to submit!
